# Shipping Optimization Challenge
Técnicas Avanzadas de Análisis de Datos
> Néstor Torres Díaz

## Objetivo 
Realizar una competición para el problema de regresión *Shipping Optimization Challenge* planteado en el siguiente [enlace](https://www.kaggle.com/datasets/salil007/1-shipping-optimization-challenge?resource=download), con el propósito de predecir el tiempo (*Shipping Time*), necesario para procesar con éxito cada envío. 

## Ficheros utilizados
Accediendo a la página de la competición través del enlace comentado en el anterior apartado, podremos ver los ficheros necesarios para participar en la misma.
- ``shipping_companies_details_1.csv``: Este fichero contiene toda la información necesaria sobre las empresas de transporte.
- ``submission_2.csv``: Este fichero debe ser presentado por los participantes. **No debe contener ningún id ni nombre de columna**. El orden de las variables objetivo debe ajustarse al formato del fichero pero la primera fila y la primera columna deberían eliminarse previas al envío. La competición solo aceptará archivos csv que consten únicamente de valores numéricos.
- ``test_2.csv``: Este fichero contiene datos históricos de envíos. El tiempo de envío es desconocido y debe predecirse.
- ``train_2_pr.csv``: Este fichero contiene envíos históricos comprendidos entre 2019 y 2020, con tiempos de envío conocidos y algunos otros detalles de los mismos, por lo que puede ser utilizado para entrenar el modelo que se plantee. Más de un registro en este fichero tiene el mismo `shipment_id`, pero su columna `shipping_company` es distinta.


## Carga de Librerías

In [1751]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as  plt

## Lectura de datos

Se cargan los conjuntos de datos y se hace una exploración inicial sobre los mismos

### Conjunto de entrenamiento

En primera instancia, se lee el conjunto de datos de entrenamiento, estableciendo el id del envío como índice y eliminando posteriormente el establecido por defecto. También se parsea la fecha almacenada en el campo `send_timestamp`. 

In [1752]:
train = pd.read_csv('data/train_2_pr.csv', index_col='shipment_id', parse_dates=['send_timestamp'])
train.drop('Unnamed: 0', axis=1, inplace=True)
train.head()

,send_timestamp,pick_up_point,drop_off_point,source_country,destination_country,freight_cost,gross_weight,shipment_charges,shipment_mode,shipping_company,selected,shipping_time
shipment_id,,,,,,,,,,,,
S000720,2019-06-08 07:17:51,A,Y,GB,IN,88.61,355.0,0.75,Air,SC3,Y,5.00741
S000725,2019-07-12 15:23:21,A,Y,GB,IN,85.65,105.0,0.90,Ocean,SC1,Y,21.41215
S000736,2019-10-04 14:23:29,A,Y,GB,IN,86.22,100.0,0.75,Air,SC3,Y,5.33692
S000738,2020-01-07 09:19:50,A,Y,GB,IN,94.43,1071.0,1.05,Air,SC2,Y,5.14792
S000739,2020-04-11 06:36:03,A,Y,GB,IN,94.24,2007.0,0.75,Air,SC3,Y,5.03067


En concreto, el conjunto de almacenamiento contiene 5114 registros de envíos para los cuales se contemplan 12 variables con información sobre los mismos.

 - **send_timestamp**: Fecha y hora en que el pedido fue enviado al país de destino, ajustada al huso horario del país de origen.
 - **pick_up_point**: Código que identifica el lugar donde se recogen las mercancías para su envío.
 - **drop_off_point**: Código que identifica el lugar donde se entregan las mercancías al final de su viaje.
 - **source_country**: País desde donde se envían las mercancías.
 - **destination_country**: País al cual se envían las mercancías.
 - **freight_cost**: Costo asociado al transporte de cada kilogramo de mercancía.
 - **gross_weight**: Peso total en kilogramos de las mercancías que se necesitan enviar.
 - **shipment_charges**: Costo fijo asociado a cada envío, independientemente del peso de la carga.
 - **shipment_mode**: Método utilizado para enviar las mercancías, como por aire, mar, etc.
 - **shipping_company**: Identificador de la compañía de envíos que es candidata para realizar el transporte de las mercancías.
 - **selected**: Indica si la compañía en `shipping_company` fue seleccionada o no para realizar el envío.
 - **shipping_time**: Duración en días que toma para que las mercancías lleguen a su destino desde el momento en que son enviadas.

Cabe añadir que el campo `shipment_id` ha sido incluído como índice, por lo tanto aquí no se referencia, pero hace alusión al identificador único del envío.

In [1753]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5114 entries, S000720 to S2082151
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   send_timestamp       5114 non-null   datetime64[ns]
 1   pick_up_point        5114 non-null   object        
 2   drop_off_point       5114 non-null   object        
 3   source_country       5114 non-null   object        
 4   destination_country  5114 non-null   object        
 5   freight_cost         5114 non-null   float64       
 6   gross_weight         5114 non-null   float64       
 7   shipment_charges     5114 non-null   float64       
 8   shipment_mode        5114 non-null   object        
 9   shipping_company     5114 non-null   object        
 10  selected             5114 non-null   object        
 11  shipping_time        5114 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 519.4+ KB


### Conjunto de Test

Con el conjunto de test, la lectura se hace de manera idéntica al conjunto de entrenamiento.

In [1754]:
test = pd.read_csv('data/test_2.csv', index_col='shipment_id', parse_dates=['send_timestamp'])
test.drop('Unnamed: 0', axis=1, inplace=True)
test.head()

,send_timestamp,pick_up_point,drop_off_point,source_country,destination_country,freight_cost,gross_weight,shipment_charges,shipment_mode,shipping_company,selected
shipment_id,,,,,,,,,,,
S002736,2019-10-04 14:27:04,A,Y,GB,IN,86.81,100.0,0.75,Air,SC3,Y
S002738,2020-01-07 09:39:35,A,Y,GB,IN,94.43,1006.0,0.75,Air,SC3,Y
S005739,2020-04-11 11:58:10,A,Y,GB,IN,93.55,321.0,1.05,Air,SC2,Y
S008722,2019-06-23 11:54:41,A,Y,GB,IN,88.74,355.0,1.05,Air,SC2,Y
S009737,2019-11-20 20:18:01,A,Y,GB,IN,92.83,115.0,1.05,Air,SC2,Y


Este conjunto está compuesto por 1260 registros para los cuales se contemplan exactamente las mismas variables que para el conjunto de entrenamiento, a excepción de la variable objetivo `shipping_time`. 

In [1755]:
test[['shipping_company','shipment_charges']].value_counts()

shipping_company  shipment_charges
SC1               0.9000              503
SC3               0.7500              421
SC2               1.0500              266
                  0.5625               53
SC1               1.1250               17
Name: count, dtype: int64

In [1756]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1260 entries, S002736 to S9443149
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   send_timestamp       1260 non-null   datetime64[ns]
 1   pick_up_point        1260 non-null   object        
 2   drop_off_point       1260 non-null   object        
 3   source_country       1260 non-null   object        
 4   destination_country  1260 non-null   object        
 5   freight_cost         1260 non-null   float64       
 6   gross_weight         1260 non-null   float64       
 7   shipment_charges     1260 non-null   float64       
 8   shipment_mode        1260 non-null   object        
 9   shipping_company     1260 non-null   object        
 10  selected             1260 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 118.1+ KB


### Conjunto de Detalles de Compañias de Transporte

Este conjunto de datos contiene información sobre las diferentes compañías de transporte que se hacen cargo de los envíos.

In [1757]:
details = pd.read_csv('data/shipping_companies_details_1.csv')
details.head()

,source_country,destination_country,shipment_mode,cut_off_time,tat,processing_days,pick_up_point,drop_off_point,min_cs,max_cs,shipping_company,shipment_charges
0,GB,IN,Ocean,12PM IST,Before CO - T+0\nAfter CO - T+1,Mon-Fri,A,Y,100,"2,500,000",SC1,0.9000
1,GB,IN,Air,24/7,Real-time,24/7,A,Y,100,"200,000",SC2,1.0500
2,GB,IN,Air,24/7,Within 15 mins,24/7,A,Y,100,"200,000",SC3,0.7500
3,GB,BD,Ocean,10 - 2 and 3 - 6 BST,Before CO - T+0\nAfter CO - T+1,Sun-Fri,A,X,50,"4,000,000",SC1,1.1250
4,GB,BD,Ocean,10 - 2 and 3 - 6 BST,Before CO - T+0\nAfter CO - T+1,Sun-Fri,A,X,50,"4,000,000",SC2,0.5625


En concreto, este conjunto de datos solo contiene 5 registros, para los cuales se tienen en cuenta 12 variables. Se puede ver como contiene muchos campos que coinciden con los vistos en los anteriores conjuntos de datos, lo cual significa que si los campos que tienen en común contienen los mismos valores se podrían llegar a combinar los conjuntos de datos para completar aún más la información.
- **source_country**: País desde donde se envían las mercancías.
- **destination_country**: País al cual se envían las mercancías.
- **shipment_mode**: Método utilizado para enviar las mercancías, como por aire, mar, etc.
- **cut_off_time**: Ventana de tiempo durante la cual las mercancías pueden ser recogidas. Por ejemplo, "10 - 2 y 3 - 6 BST" indica que la recogida está disponible entre las 10 a.m. y las 2 p.m., y nuevamente entre las 3 p.m. y las 6 p.m., todas las horas en Horario Estándar de Bangladesh.
- **tat**: Tiempo que toma desde que se realiza el pedido del paquete hasta su envío. "T" se refiere a la hora en que se envió el paquete, y "CO" significa hora de corte, o la hora después de la cual el paquete ya no puede ser enviado. "Before CO - T+0" y "After CO - T+1" significa que si las mercancías se envían a la compañía de transporte antes de la hora de corte, entonces el paquete será enviado el mismo día, de lo contrario será enviado al día siguiente.
- **processing_days**: Días en los que el punto de recogida está operativo.
- **pick_up_point**: Código que identifica el lugar donde se recogen las mercancías para su envío.
- **drop_off_point**: Código que identifica el lugar donde se entregan las mercancías al final de su viaje.
- **min_cs**: Costo mínimo asociado con el envío de mercancías.
- **max_cs**: Costo máximo asociado con el envío de mercancías.
- **shipping_company**: Identificador de la compañía de envíos que es candidata para realizar el transporte de las mercancías.
- **shipment_charges**: Costo fijo asociado a cada envío, independientemente del peso de la carga.

In [1758]:
details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   source_country       5 non-null      object 
 1   destination_country  5 non-null      object 
 2   shipment_mode        5 non-null      object 
 3   cut_off_time         5 non-null      object 
 4   tat                  5 non-null      object 
 5   processing_days      5 non-null      object 
 6   pick_up_point        5 non-null      object 
 7   drop_off_point       5 non-null      object 
 8   min_cs               5 non-null      int64  
 9   max_cs               5 non-null      object 
 10  shipping_company     5 non-null      object 
 11  shipment_charges     5 non-null      float64
dtypes: float64(1), int64(1), object(10)
memory usage: 612.0+ bytes


## Integración de Conjuntos de Datos
En este apartado se integran los datos de detalles de las compañías con los conjuntos de entrenamiento y de test, de manera que la información esté lo más completa posible.

### Conjunto de entrenamiento
En concreto, hemos visto que los campos que tienen en común el conjunto de entrenamiento y el conjunto de detalles de las compañías de transporte son los campos:
 - `source_country`
 - `destination_country`
 - `shipment_mode`
 - `pick_up_point`
 - `drop_off_point`
 - `shipping_company`
 - `shipment_charges`

Por lo tanto, se comprueba que los valores presentados para dichos campos en ambos conjuntos de datos, no contengan ningún tipo de espacio o símbolo no deseado o que al menos manejen los mismos valores.

In [1759]:
common_fields = ['source_country', 'destination_country', 'shipment_mode', 'pick_up_point', 'drop_off_point', 'shipping_company', 'shipment_charges']

In [1760]:
def check_for_differences(a_df, b_df, fields):
    for field in fields:
        unique_a_df = set(a_df[field].unique())
        unique_b_df = set(b_df[field].unique())

        a_minus_b = unique_a_df - unique_b_df
        b_minus_a = unique_b_df - unique_a_df
        if (len(a_minus_b) > 0):
            print(field, "- Valores en a que no están en b ->", a_minus_b)
        if (len(b_minus_a) > 0):
            print(field,"- Valores en b que no están en a ->", b_minus_a)
        if ((len(b_minus_a) == 0) and (len(a_minus_b) == 0)):
            print(field,"- Manejan los mismos valores ->", unique_a_df)

In [1761]:
check_for_differences(train, details, common_fields)

source_country - Manejan los mismos valores -> {'GB'}
destination_country - Manejan los mismos valores -> {'IN', 'BD'}
shipment_mode - Manejan los mismos valores -> {'Ocean', 'Air'}
pick_up_point - Manejan los mismos valores -> {'A'}
drop_off_point - Valores en a que no están en b -> {'X', 'Y'}
drop_off_point - Valores en b que no están en a -> {' Y ', ' X '}
shipping_company - Manejan los mismos valores -> {'SC1', 'SC3', 'SC2'}
shipment_charges - Manejan los mismos valores -> {0.75, 0.9, 1.125, 1.05, 0.5625}


Al hacer estas comprobaciones nos encontramos con que los valores utilizados en el conjunto de datos `details` para el campo `drop_off_point` difiere de los valores utilizados en el conjunto de `train`. En concreto vemos que hay espacios por delante y detrás, así que procedemos a eliminarlos.

In [1762]:
details['drop_off_point'] = details['drop_off_point'].str.strip()
check_for_differences(train, details, ['drop_off_point'])

drop_off_point - Manejan los mismos valores -> {'X', 'Y'}


Ahora que sabemos que manejan los mismos valores, procedemos a combinarlos en un único conjunto de entrenamiento que contendrá también la información de detalles sobre las compañias de transporte.

In [1763]:

train = pd.merge(train, details, on=common_fields, how='inner')
train.head()

,send_timestamp,pick_up_point,drop_off_point,source_country,destination_country,freight_cost,gross_weight,shipment_charges,shipment_mode,shipping_company,selected,shipping_time,cut_off_time,tat,processing_days,min_cs,max_cs
0,2019-06-08 07:17:51,A,Y,GB,IN,88.61,355.0,0.75,Air,SC3,Y,5.00741,24/7,Within 15 mins,24/7,100,"200,000"
1,2019-10-04 14:23:29,A,Y,GB,IN,86.22,100.0,0.75,Air,SC3,Y,5.33692,24/7,Within 15 mins,24/7,100,"200,000"
2,2020-04-11 06:36:03,A,Y,GB,IN,94.24,2007.0,0.75,Air,SC3,Y,5.03067,24/7,Within 15 mins,24/7,100,"200,000"
3,2019-05-27 11:53:23,A,Y,GB,IN,87.84,228.7,0.75,Air,SC3,Y,5.18611,24/7,Within 15 mins,24/7,100,"200,000"
4,2019-06-08 07:46:01,A,Y,GB,IN,89.00,119.0,0.75,Air,SC3,Y,5.22060,24/7,Within 15 mins,24/7,100,"200,000"


### Conjunto de test
En concreto, hemos visto que los campos que tienen en común el conjunto de test y el conjunto de detalles de las compañías de transporte son los mismos campos y por lo tanto, se comprueba también que los valores presentados para dichos campos en ambos conjuntos de datos, no contengan ningún tipo de espacio o símbolo no deseado o que al menos manejen los mismos valores.

In [1764]:
check_for_differences(test, details, common_fields)

source_country - Manejan los mismos valores -> {'GB'}
destination_country - Manejan los mismos valores -> {'IN', 'BD'}
shipment_mode - Manejan los mismos valores -> {'Ocean', 'Air'}
pick_up_point - Manejan los mismos valores -> {'A'}
drop_off_point - Manejan los mismos valores -> {'X', 'Y'}
shipping_company - Manejan los mismos valores -> {'SC1', 'SC3', 'SC2'}
shipment_charges - Manejan los mismos valores -> {0.75, 1.05, 1.125, 0.5625, 0.9}


En este caso nos encontramos con que ambos conjuntos de datos manejan los mismos valores y además estos no contienen ningún espacio ni símbolo no deseado, por lo que podemos proceder a integrar los datos de detalles de las compañías de transporte con el conjunto de test, dejando un nuevo conjunto de test más completo.

In [1765]:
test = pd.merge(test, details, on=common_fields, how='inner')
test.head()

,send_timestamp,pick_up_point,drop_off_point,source_country,destination_country,freight_cost,gross_weight,shipment_charges,shipment_mode,shipping_company,selected,cut_off_time,tat,processing_days,min_cs,max_cs
0,2019-10-04 14:27:04,A,Y,GB,IN,86.81,100.0,0.75,Air,SC3,Y,24/7,Within 15 mins,24/7,100,"200,000"
1,2020-01-07 09:39:35,A,Y,GB,IN,94.43,1006.0,0.75,Air,SC3,Y,24/7,Within 15 mins,24/7,100,"200,000"
2,2019-11-20 20:30:05,A,Y,GB,IN,92.48,115.0,0.75,Air,SC3,Y,24/7,Within 15 mins,24/7,100,"200,000"
3,2020-04-11 14:53:25,A,Y,GB,IN,94.45,325.0,0.75,Air,SC3,Y,24/7,Within 15 mins,24/7,100,"200,000"
4,2019-03-23 06:39:22,A,Y,GB,IN,90.39,61.0,0.75,Air,SC3,Y,24/7,Within 15 mins,24/7,100,"200,000"


## Análisis Exploratorio de Datos (EDA)

### Análisis descriptivo

Puesto que se ha llevado a cabo la integración de los datos de entrenamiento y de test con el conjunto de detalles sobre compañias de transporte, el análisis se centrará principalmente en los conjuntos resultantes de esa combinación, más concretamente en el de entrenamiento. Aún así, cabe destacar que la variable "shipping_time" sólo está en el conjunto de entrenamiento.

In [1766]:
field_differences = set(train.columns) - set(test.columns)
print("Campos presentes en el conjunto de entrenamiento que no están en el de test: ", field_differences)

Campos presentes en el conjunto de entrenamiento que no están en el de test:  {'shipping_time'}


Comprobamos nuevamente la información de este conjunto de entrenamiento, donde recordamos lo que representa cada variable:

 - **send_timestamp**: Fecha y hora en que el pedido fue enviado al país de destino, ajustada al huso horario del país de origen.
 - **pick_up_point**: Código que identifica el lugar donde se recogen las mercancías para su envío.
 - **drop_off_point**: Código que identifica el lugar donde se entregan las mercancías al final de su viaje.
 - **source_country**: País desde donde se envían las mercancías.
 - **destination_country**: País al cual se envían las mercancías.
 - **freight_cost**: Costo asociado al transporte de cada kilogramo de mercancía.
 - **gross_weight**: Peso total en kilogramos de las mercancías que se necesitan enviar.
 - **shipment_charges**: Costo fijo asociado a cada envío, independientemente del peso de la carga.
 - **shipment_mode**: Método utilizado para enviar las mercancías, como por aire, mar, etc.
 - **shipping_company**: Identificador de la compañía de envíos que es candidata para realizar el transporte de las mercancías.
 - **selected**: Indica si la compañía en `shipping_company` fue seleccionada o no para realizar el envío.
 - **shipping_time**: Duración en días que toma para que las mercancías lleguen a su destino desde el momento en que son enviadas.
 - **cut_off_time**: Ventana de tiempo durante la cual las mercancías pueden ser recogidas. Por ejemplo, "10 - 2 y 3 - 6 BST" indica que la recogida está disponible entre las 10 a.m. y las 2 p.m., y nuevamente entre las 3 p.m. y las 6 p.m., todas las horas en Horario Estándar de Bangladesh.
- **tat**: Tiempo que toma desde que se realiza el pedido del paquete hasta su envío. "T" se refiere a la hora en que se envió el paquete, y "CO" significa hora de corte, o la hora después de la cual el paquete ya no puede ser enviado. "Before CO - T+0" y "After CO - T+1" significa que si las mercancías se envían a la compañía de transporte antes de la hora de corte, entonces el paquete será enviado el mismo día, de lo contrario será enviado al día siguiente.
- **processing_days**: Días en los que el punto de recogida está operativo.
- **min_cs**: Costo mínimo asociado con el envío de mercancías.
- **max_cs**: Costo máximo asociado con el envío de mercancías.

Cabe añadir que el campo `shipment_id` ha sido incluído como índice, por lo tanto aquí no se referencia, pero hace alusión al identificador único del envío.

Por un lado, vemos que *no se encuentran registros nulos* para ninguna de las variables del dataset y además se observa algún que otro *tipo de datos que está mal asignado*, como puede ser `max_cs` que representa el costo máximo asociado al envío, es decir, un valor numérico y está siendo tratado como un `object` mientras que el costo mínimo asociado al envío, que debería tener el mismo formato numérico, está siendo tratado como un `int64`. 

In [1767]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5114 entries, 0 to 5113
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   send_timestamp       5114 non-null   datetime64[ns]
 1   pick_up_point        5114 non-null   object        
 2   drop_off_point       5114 non-null   object        
 3   source_country       5114 non-null   object        
 4   destination_country  5114 non-null   object        
 5   freight_cost         5114 non-null   float64       
 6   gross_weight         5114 non-null   float64       
 7   shipment_charges     5114 non-null   float64       
 8   shipment_mode        5114 non-null   object        
 9   shipping_company     5114 non-null   object        
 10  selected             5114 non-null   object        
 11  shipping_time        5114 non-null   float64       
 12  cut_off_time         5114 non-null   object        
 13  tat                  5114 non-nul

Este conjunto de datos está compuesto por 5114 registros y 17 variables.

In [1768]:
train.shape

(5114, 17)

Se comprueban también la cantidad de valores únicos para cada variable, de manera que podamos distinguir la cantidad de valores distintos que puede tomar cada una de ellas.

In [1769]:
train.nunique()

send_timestamp         4804
pick_up_point             1
drop_off_point            2
source_country            1
destination_country       2
freight_cost           2000
gross_weight           1301
shipment_charges          5
shipment_mode             2
shipping_company          3
selected                  1
shipping_time          4315
cut_off_time              3
tat                       3
processing_days           3
min_cs                    2
max_cs                    3
dtype: int64

En este caso vemos que las variables `pick_up_point`, `source_country` y `selected` sólo toman un valor, lo que podría introducir ruido en nuestro conjunto de datos al considerarse información redundante.

El resto de variables que toman de de 2 a 3 valores distintos habría que analizarlas más en profundidad para ver su frecuencia y posiblemente tratarlas como variables categóricas.


### Análisis de duplicados

### Análisis de valores faltantes

### Visualización de datos

#### Histogramas

#### Diagramas de barras

#### Correlación entre variables

#### Boxplots